In [26]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from xgboost import XGBClassifier
import re
from sklearn.metrics import accuracy_score,confusion_matrix

In [27]:
df = pd.read_parquet(r"C:\Users\yusuf\Desktop\Workspace url\fake_news.parquet")

In [28]:
def clean_m(x):
    x = x.strip().lower()
    x = re.sub('\(.+\)','',x)
    x = re.sub('\\n', '', x)
    x = re.sub('[^1-9a-zA-Z ]', '', x)
    return x

In [29]:
df['text'] = df.text.apply(lambda x: clean_m(str(x)))
df['text']=df.text.astype('str')

In [30]:
vec = TfidfVectorizer()

x = vec.fit_transform(df.text)
y = df.label

In [31]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=12, test_size=0.2)

In [32]:
modelx = LogisticRegression()
params = {'penalty':['l2'],
          'C':[1.2],
          'max_iter':[1000]}
cvb = StratifiedKFold(n_splits=5)

grid = GridSearchCV(modelx, param_grid=params,cv=cvb, verbose=5,scoring = 'accuracy')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..C=1.2, max_iter=1000, penalty=l2;, score=0.814 total time=   9.5s
[CV 2/5] END ..C=1.2, max_iter=1000, penalty=l2;, score=0.817 total time=  10.7s
[CV 3/5] END ..C=1.2, max_iter=1000, penalty=l2;, score=0.804 total time=   8.1s
[CV 4/5] END ..C=1.2, max_iter=1000, penalty=l2;, score=0.808 total time=  11.5s
[CV 5/5] END ..C=1.2, max_iter=1000, penalty=l2;, score=0.806 total time=   8.7s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(),
             param_grid={'C': [1.2], 'max_iter': [1000], 'penalty': ['l2']},
             scoring='accuracy', verbose=5)

In [48]:
y_pred = grid.predict(x_test)
print(accuracy_score(y_pred,y_test))
confusion_matrix(y_pred,y_test)

0.8059701492537313


array([[2006,  323],
       [ 730, 2368]], dtype=int64)

In [39]:
pickle.dump(grid, open('model_a.sav','wb'))
pickle.dump(vec, open('vector.sav','wb'))

In [47]:
kk = 'who is there'
kk = clean_m(kk)
arr=vec.transform([kk])
result = grid.predict(arr).tolist()
result[0]


1

In [25]:
df.dtypes

text     object
label     int64
dtype: object